# Libraries

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
# Load from an csv file
df = pd.read_csv('../data/fps-dataset.csv', low_memory=False)
df

,CpuName,CpuNumberOfCores,CpuNumberOfThreads,CpuBaseClock,CpuCacheL1,CpuCacheL2,CpuCacheL3,CpuFrequency,CpuMultiplier,CpuMultiplierUnlocked,CpuProcessSize,CpuTDP,CpuTurboClock,GpuName,GpuArchitecture,GpuBandwidth,GpuBaseClock,GpuBoostClock,GpuBus nterface,GpuDieSize,GpuDirectX,GpuFP32Performance,GpuMemoryBus,GpuMemorySize,GpuMemoryType,GpuOpenCL,GpuOpenGL,GpuPixelRate,GpuProcessSize,GpuNumberOfROPs,GpuShaderModel,GpuNumberOfShadingUnits,GpuNumberOfTMUs,GpuTextureRate,GpuNumberOfTransistors,GpuVulkan,GameName,GameResolution,GameSetting,Dataset,FPS
0,Intel Core i7-920,4.0,8.0,133.0,256.0,1024.0,8.0,2666.0,20.0,0,45.0,130.0,2933.0,AMD Radeon RX 480,GCN 4.0,256000.0,1120.0,1266.0,PCIe 3.0 x16,0.000232,12,5834000.0,256.0,8000.0,GDDR5,2,4.6,40510.0,14.0,32.0,6.4,2304.0,144.0,182300.0,5700.0,1.2.131,counterStrikeGlobalOffensive,1080.0,low,userbenchmark,70.0
1,Intel Core i7-920,4.0,8.0,133.0,256.0,1024.0,8.0,2666.0,20.0,0,45.0,130.0,2933.0,AMD Radeon RX 480,GCN 4.0,256000.0,1120.0,1266.0,PCIe 3.0 x16,0.000232,12,5834000.0,256.0,8000.0,GDDR5,2,4.6,40510.0,14.0,32.0,6.4,2304.0,144.0,182300.0,5700.0,1.2.131,counterStrikeGlobalOffensive,1080.0,low,userbenchmark,230.0
2,Intel Core i5-4690,4.0,4.0,100.0,256.0,1024.0,6.0,3400.0,34.0,0,22.0,84.0,3800.0,NVIDIA GeForce GTX 1080 11Gbps,Pascal,352300.0,1607.0,1733.0,PCIe 3.0 x16,0.000314,12,8873000.0,256.0,8000.0,GDDR5X,1.2,4.6,110900.0,16.0,64.0,6.4,2560.0,160.0,277300.0,7200.0,1.2.131,counterStrikeGlobalOffensive,1080.0,low,userbenchmark,250.0
3,Intel Core i5-4690,4.0,4.0,100.0,256.0,1024.0,6.0,3400.0,34.0,0,22.0,84.0,3800.0,NVIDIA GeForce GTX 1080 11Gbps,Pascal,352300.0,1607.0,1733.0,PCIe 3.0 x16,0.000314,12,8873000.0,256.0,8000.0,GDDR5X,1.2,4.6,110900.0,16.0,64.0,6.4,2560.0,160.0,277300.0,7200.0,1.2.131,counterStrikeGlobalOffensive,1080.0,low,userbenchmark,300.0
4,Intel Core i5-4690,4.0,4.0,100.0,256.0,1024.0,6.0,3400.0,34.0,0,22.0,84.0,3800.0,AMD Radeon R7 260X,GCN 2.0,104000.0,1100.0,1100.0,PCIe 3.0 x16,0.000160,12,1971000.0,128.0,2000.0,GDDR5,2,4.6,17600.0,28.0,16.0,6.3,896.0,56.0,61600.0,2080.0,1.2.131,counterStrikeGlobalOffensive,1080.0,low,userbenchmark,200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404437,AMD Ryzen 5 3600,6.0,12.0,100.0,576.0,3072.0,32.0,3600.0,36.0,1,7.0,65.0,4200.0,AMD Radeon RX 570,GCN 4.0,224000.0,1168.0,1244.0,PCIe 3.0 x16,0.000232,12,5095000.0,256.0,4000.0,GDDR5,2,4.6,39810.0,14.0,32.0,6.4,2048.0,128.0,159200.0,5700.0,1.2.131,rainbowSixSiege,1080.0,max,fpsbenchmark,51.5
404438,AMD Ryzen 5 2600X,6.0,12.0,100.0,576.0,3072.0,16.0,3600.0,36.0,1,12.0,95.0,4250.0,AMD Radeon RX 570,GCN 4.0,224000.0,1168.0,1244.0,PCIe 3.0 x16,0.000232,12,5095000.0,256.0,4000.0,GDDR5,2,4.6,39810.0,14.0,32.0,6.4,2048.0,128.0,159200.0,5700.0,1.2.131,rainbowSixSiege,1080.0,max,fpsbenchmark,49.9
404439,AMD Ryzen 5 1600X,6.0,12.0,100.0,576.0,3072.0,16.0,3600.0,36.0,1,14.0,95.0,4000.0,AMD Radeon RX 570,GCN 4.0,224000.0,1168.0,1244.0,PCIe 3.0 x16,0.000232,12,5095000.0,256.0,4000.0,GDDR5,2,4.6,39810.0,14.0,32.0,6.4,2048.0,128.0,159200.0,5700.0,1.2.131,rainbowSixSiege,1080.0,max,fpsbenchmark,46.9
404440,AMD Ryzen 5 2600,6.0,12.0,100.0,576.0,3072.0,16.0,3400.0,34.0,1,12.0,65.0,3900.0,AMD Radeon RX 570,GCN 4.0,224000.0,1168.0,1244.0,PCIe 3.0 x16,0.000232,12,5095000.0,256.0,4000.0,GDDR5,2,4.6,39810.0,14.0,32.0,6.4,2048.0,128.0,159200.0,5700.0,1.2.131,rainbowSixSiege,1080.0,max,fpsbenchmark,46.4


In [3]:
# Showing basic information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404442 entries, 0 to 404441
Data columns (total 41 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   CpuName                  404442 non-null  object 
 1   CpuNumberOfCores         404442 non-null  float64
 2   CpuNumberOfThreads       404442 non-null  float64
 3   CpuBaseClock             404442 non-null  float64
 4   CpuCacheL1               404442 non-null  float64
 5   CpuCacheL2               404442 non-null  float64
 6   CpuCacheL3               404442 non-null  float64
 7   CpuFrequency             404442 non-null  float64
 8   CpuMultiplier            404442 non-null  float64
 9   CpuMultiplierUnlocked    404442 non-null  object 
 10  CpuProcessSize           404442 non-null  float64
 11  CpuTDP                   404442 non-null  float64
 12  CpuTurboClock            404442 non-null  float64
 13  GpuName                  404442 non-null  object 
 14  GpuA

# Feature Engineering

## André Pilco

### GPU to CPU ratio
In order to identify bottlenecks or imbalances, we calculate the ratio of GPU to CPU.

In [4]:
df['GpuCpuRatio'] = df.apply(lambda row: row['GpuBoostClock'] / row['CpuTurboClock'], axis = 1)

### Cache to Core ratio
Analyzing the ratio of CPU cache size (L1, L2 and L3) to the number of cores

In [5]:
df['CacheCoreRatio'] = df.apply(lambda row: (row['CpuCacheL1'] + row['CpuCacheL2'] + row['CpuCacheL3']) / row['CpuNumberOfCores'], axis = 1)

### Overclocking potential
Compute the potentital overclocking headrrom.

In [6]:
df['CpuOverclockPotential'] = df.apply(lambda row: (row['CpuTurboClock'] - row['CpuBaseClock']) / row['CpuBaseClock'], axis = 1)

## John Sovero

### EffectiveMemory
To determine how efficient a GPU is in terms of its computing capabilities

In [7]:
df['EffectiveMemory'] = df['GpuMemoryBus'] * df['GpuBandwidth']

### PerformanceClockRatio
To evaluate the ability of a CPU or GPU to handle high-performance tasks and to predict its behavior in high-demand situations.

In [8]:
df['PerformanceClockRatio'] = df['CpuTurboClock'] / df['GpuBoostClock']

### PerformancePerWatt
To capture how the most energy-efficient GPUs tend to maintain high performance per watt consumed

In [9]:
# GPU energy efficiency
df['PerformancePerWatt'] = df['GpuFP32Performance'] / df['GpuNumberOfTransistors']

## Iam Alvarez

### Ratio of Cores to CPU Threads
Measures the ratio between the number of physical cores and CPU threads to assess parallel processing capability.

In [17]:
df['CpuCoreThreadRatio'] = df['CpuNumberOfCores'] / df['CpuNumberOfThreads']

### CPU Power Index
It combines the turbo clock speed, the number of cores and the size of the L3 cache to evaluate the total CPU power.

In [18]:
df['CpuPowerIndex'] = df['CpuTurboClock'] * df['CpuNumberOfCores'] * df['CpuCacheL3']

### GPU Average Frequency Rating
Calculates the average GPU frequency to represent its overall operating performance.

In [19]:
df['GpuAverageClock'] = (df['GpuBaseClock'] + df['GpuBoostClock']) / 2

In [20]:
df.head()

,CpuName,CpuNumberOfCores,CpuNumberOfThreads,CpuBaseClock,CpuCacheL1,CpuCacheL2,CpuCacheL3,CpuFrequency,CpuMultiplier,CpuMultiplierUnlocked,CpuProcessSize,CpuTDP,CpuTurboClock,GpuName,GpuArchitecture,GpuBandwidth,GpuBaseClock,GpuBoostClock,GpuBus nterface,GpuDieSize,GpuDirectX,GpuFP32Performance,GpuMemoryBus,GpuMemorySize,GpuMemoryType,GpuOpenCL,GpuOpenGL,GpuPixelRate,GpuProcessSize,GpuNumberOfROPs,GpuShaderModel,GpuNumberOfShadingUnits,GpuNumberOfTMUs,GpuTextureRate,GpuNumberOfTransistors,GpuVulkan,GameName,GameResolution,GameSetting,Dataset,FPS,CpuCoreThreadRatio,CpuPowerIndex,GpuAverageClock
0,Intel Core i7-920,4.0,8.0,133.0,256.0,1024.0,8.0,2666.0,20.0,0,45.0,130.0,2933.0,AMD Radeon RX 480,GCN 4.0,256000.0,1120.0,1266.0,PCIe 3.0 x16,0.000232,12,5834000.0,256.0,8000.0,GDDR5,2,4.6,40510.0,14.0,32.0,6.4,2304.0,144.0,182300.0,5700.0,1.2.131,counterStrikeGlobalOffensive,1080.0,low,userbenchmark,70.0,0.5,93856.0,1193.0
1,Intel Core i7-920,4.0,8.0,133.0,256.0,1024.0,8.0,2666.0,20.0,0,45.0,130.0,2933.0,AMD Radeon RX 480,GCN 4.0,256000.0,1120.0,1266.0,PCIe 3.0 x16,0.000232,12,5834000.0,256.0,8000.0,GDDR5,2,4.6,40510.0,14.0,32.0,6.4,2304.0,144.0,182300.0,5700.0,1.2.131,counterStrikeGlobalOffensive,1080.0,low,userbenchmark,230.0,0.5,93856.0,1193.0
2,Intel Core i5-4690,4.0,4.0,100.0,256.0,1024.0,6.0,3400.0,34.0,0,22.0,84.0,3800.0,NVIDIA GeForce GTX 1080 11Gbps,Pascal,352300.0,1607.0,1733.0,PCIe 3.0 x16,0.000314,12,8873000.0,256.0,8000.0,GDDR5X,1.2,4.6,110900.0,16.0,64.0,6.4,2560.0,160.0,277300.0,7200.0,1.2.131,counterStrikeGlobalOffensive,1080.0,low,userbenchmark,250.0,1.0,91200.0,1670.0
3,Intel Core i5-4690,4.0,4.0,100.0,256.0,1024.0,6.0,3400.0,34.0,0,22.0,84.0,3800.0,NVIDIA GeForce GTX 1080 11Gbps,Pascal,352300.0,1607.0,1733.0,PCIe 3.0 x16,0.000314,12,8873000.0,256.0,8000.0,GDDR5X,1.2,4.6,110900.0,16.0,64.0,6.4,2560.0,160.0,277300.0,7200.0,1.2.131,counterStrikeGlobalOffensive,1080.0,low,userbenchmark,300.0,1.0,91200.0,1670.0
4,Intel Core i5-4690,4.0,4.0,100.0,256.0,1024.0,6.0,3400.0,34.0,0,22.0,84.0,3800.0,AMD Radeon R7 260X,GCN 2.0,104000.0,1100.0,1100.0,PCIe 3.0 x16,0.000160,12,1971000.0,128.0,2000.0,GDDR5,2,4.6,17600.0,28.0,16.0,6.3,896.0,56.0,61600.0,2080.0,1.2.131,counterStrikeGlobalOffensive,1080.0,low,userbenchmark,200.0,1.0,91200.0,1100.0


In [21]:
df.to_csv('../data/fps-dataset-clean.csv', index=False)